# IMPORTAMOS LAS LIBRERIAS 

In [ ]:
import numpy as np
import pandas as pd
import json
import re
pd.set_option('display.max_rows', 500) # para visualizar todo el DataFrame


In [ ]:
db = np.load("BD.npy",allow_pickle=True) 

# Volvemos el np.array un DataFrame
df = pd.DataFrame(db)
df.drop([0,1,2],axis=0,inplace=True)
df.reset_index(inplace=True,drop=True)

# Le damos nombres a las columnas
columnas = ['id','created_at','update_at','dalete_at','tipo','folio','area','subarea','texto','valor','document_id']
df.columns = columnas

In [ ]:
tipos =  list(df['tipo'].unique())

In [ ]:
df[df['document_id']==360].sort_values('tipo')
df[df['tipo']==11][:50]

In [ ]:
tipos_area = {}
for i in tipos:
    unicos = df[df['tipo']==i]['area'].unique()
    tipos_area[i] = list(unicos)

In [ ]:
tipos_area

In [ ]:
t_21 = {
    "Inmunohistoquímica":5,
    "Citometría de flujo":6,
    "Clínica exclusivamente":7,
    "Otro":8,
    "Genética":9,
    "Patología básica":10,
    "Desconocido":99,
    "Persona con aseguramiento":55
}

In [ ]:
t_22 = {
    "Clínica, usuario con coagulopatía":1,
    "Clínica, debido a localización del tumor":2,
    "Clínica, debido al estado funcional del usuario (deterioro)":3,
    "Negativa realizar estudio":4,
    "Administrativa":5,
    "Clínica por reporte de imágenes o laboratorios":6,
    "Persona con aseguramiento":55,
    "Tiene confirmación por histopatología":98,
    "Desconocido":99
}

In [ ]:
t_25 = {
    "codigo IPS": None ,
    "Persona con aseguramiento":55,
    "Diagnóstico fuera del país":96,
    "Desconocido":99
}

In [ ]:
t_27 = { # enfermedad actual
    "Adenocarcinoma":1,
    "Carcinoma escamocelular":2,
    "Carcinoma de células basales":3,
    "Carcinoma diferente":4,
    "Oligodendroglioma":5,
    "Astrocitoma":6,
    "Ependimoma":7,
    "Neuroblastoma":8,
    "Meduloblastoma":9,
    "Hepatoblastoma":10,
    "Rabdomiosarcoma":11,
    "Leiomiosarcoma":12,
    "Osteosarcoma":13,
    "Fibrosarcoma":14,
    "Angiosarcoma":15,
    "Condrosarcoma":16,
    "Otros sarcomas":17,
    "Pancreatoblastoma":18,
    "Blastoma pleuropulmonar":19,
    "Otros tipos histológicos no mencionados":20,
    "Melanoma":21,
    "Carcinoma papilar de tiroides":24,
    "Persona con aseguramiento":25,
    "No se realizó estudio histopatológico":98
}

In [ ]:
t_28 = {
    "grado 1":1,
    "grado 2":2,
    "grado 3":4,
    "Anaplásico":4,
    "Persona con aseguramiento":55
    "cáncer sólido":94,
    "No es sólido":95,
    "No se realizó estudio histopatológico":98,
    
}

In [ ]:
t_29 = {

}

"""
CÁNCER DE MAMA Y CÁNCER GÁSTRICO 15 0= estadio clínico (ec) 0 (tumor in situ) 2= ec IA o 1A 5= ec IB o 1b 11= ec IIA o 2a 14= ec IIB 17= ec IIIA o 3a 18= ec IIIB o 3b 19= ec IIIC o 3c 20= ec IV o 4 CANCER DE PRÓSTATA 7a Edición TNM 0= estadio clínico (ec) o (tumor in situ) 1= ec I o 1 11= ec IIA o 2a 14= ec IIB 16= ec III o 3 20= ec IV o 4 8ª Edición TNM 15= ec IIC o 2c 17= ec IIIA o 3a 18= ec IIIB o 3b 19= ec IIIC o 3c 21= ec IVA o 4a 22= ec IVB o 4b

 CANCER DE PULMON 7a Edición TNM 0= estadio clínico (ec) o (tumor in situ) 2= ec IA o 1A 5= ec IB o 1b 11= ec IIA o 2a 14= ec IIB 17= ec IIIA o 3a 18= ec IIIB o 3b 20= ec IV o 4 8ª Edición TNM 3= ec IA1 4= ec IA2 19= ec IIIC o 3c 21= ec IVA o 4a 22= ec IVB o 4b 16 26= Estadio IAB MELANOMA 0= estadio clínico (ec) o (tumor in situ) 2= ec IA o 1A 5= ec IB o 1b 11= ec IIA o 2a 14= ec IIB 15= ec IIC o 2c 16= ec III o 3 17= ec IIIA o 3a 18= ec IIIB o 3b 19= ec IIIC o 3c 20= ec IV o 4 29= ec IIID o 3d
 
CÁNCER DE COLON Y RECTO 0= estadio clínico (ec) o (tumor in situ) 1= ec I o 1 11= ec IIA o 2a 14= ec IIB 15= ec IIC o 2c 17= ec IIIA o 3a 18= ec IIIB o 3b 19= ec IIIC o 3c 21= ec IVA o 4a 22= ec IVB o 4b 

CÁNCER ANAL (Agrupador Colon y recto) 0= estadio clínico (ec) o (tumor in situ) 1= ec I o 1 10= ec II o 2 17= ec IIIA o 3a 18= ec IIIB o 3b 20= ec IV o 4 8ª Edición TNM 1= ec I o 1 11= ec IIA o 2a 14= ec IIB o 2b 17= ec IIIA o 3a 18= ec IIIB o 3b 19= ec IIIC o 3c 20=ec IV o 4 17
 
CÁNCER DE CÉRVIX (FIGO) 0= estadio clínico (ec) o (tumor in situ) 1= ec I o 1 2= ec IA o 1A 3= ec IA1 4= ec IA2 5= ec IB o 1b 6= ec IB1 7= ec IB2 10= ec II o 2 11= ec IIA o 2a 12= ec IIA1 13= ec IIA2 14= ec IIB 16= ec III o 3 17= ec IIIA o 3a 18= ec IIIB o 3b 19= ec IIIC o 3c 21= ec IVA o 4a 22= ec IVB o 4b 27= ec IIIC1 o 3c1 28= ec IIIC2 o 3c2 30= ec IB3 31= ec IC1 32= ec IC2 33= ec IC3 OTRAS OPCIONES 8= ec IC o 1c 9= ec IS o 1s 23= ec IVC o 4c 24= ec 4S (para neuroblastoma) 25= ec V o 5 55= Persona con aseguramiento (régimen subsidiado o contributivo y que no son PPNA) que recibió servicios de salud por parte del ente territorial durante el periodo de reporte. 98= No Aplica (Es cáncer de piel basocelular, es cáncer hematológico o es cáncer en SNC, excepto neuroblastoma). 99= Desconocido, el dato de esta variable no se encuentra descrito en los soportes clínicos.
"""

In [ ]:
t_31 = {
    "Sí se le realizó":1,
    "No se le realizó":2,
    "Persona con aseguramiento":55,
    "No Aplica porque es cáncer de mama in situ":97,
    "No Aplica, no es cancer de mama":98,
    "Desconocido":19
}

In [ ]:
t_33 = {
    "positivo":1,
    "indeterminado":2,
    "negativo":3,
    "cero":4,
    "Persona con aseguramiento":55,
    "No Aplica porque es cáncer de mama in situ":97,
    "No aplica no es cáncer de mama":98,
    "Desconocido":99
}

In [ ]:
t_36 = {
    " Estadio I":1,
    " Estadio II":2,
    " Estadio III":3,
    " Estadio IV":4,
    " Estadio IA":5,
    " Estadio IB":6,
    " Estadio IIA":7,
    " Estadio IIB":8,
    " Estadio IIIA":9,
    " Estadio IIIB":10,
    " Estadio IVA":11,
    " Estadio IVB":12,
    " Extranodal cualquier estadio":13,
    " Primario SNC":14,
    " Primario Mediastinal":15,
    " Primario de otros órganos":16,
    "Persona con aseguramiento":55,
    "No Aplica, cancer diferente":98,
    "Desconocido":99
}

In [ ]:
t_37 = {

}
"""
 Registre la opción que corresponde a la clasificación Gleason.
11= Gleason ≤ 6: ≤ 3+3
12= Gleason 7: 3+4
13= Gleason 7: 4+3
14= Gleason 8: 4+4 o 3+5 o 5+3
15= Gleason 9 o 10: 4+5 o 5+4 o 5+5

97= Es cáncer de próstata, pero no hay información acerca de esta estadificación porque el
diagnóstico fue clínico.
98= No es cáncer de próstata.
99= Es cáncer de próstata, pero no hay información en la historia clínica acerca de esta clasificación
a pesar de que fue diagnóstico histopatológico.
55= Persona con aseguramiento (régimen subsidiado o contributivo y que no son PPNA) que recibió
servicios de salud por parte del ente territorial durante el periodo de reporte.
 """

In [ ]:
t_38 = {
    "Bajo riesgo":1,
    "Riesgo intermedio bajo":2,
    "Riesgo intermedio":3,
    "Riesgo intermedio alto":4,
    "Riesgo alto":5,
    "Persona con aseguramiento":55,
    "No Aplica":98,
    "Desconocido":99
}


In [ ]:
t_40 = {
    "Curación":1,
    "Paliación":2,
    "Persona con aseguramiento":55,
    "Desconocido":99
}


In [ ]:
t_41 = {
    "Observación previa a tratamiento":1,
    "Ofrecer tratamiento curativo":2,
    "Observación o seguimiento":3,
    "observacion y curación":4,
    "curación y seguimiento":5,
    "observación, curación y seguimiento":6,
    "Persona con aseguramiento":55,
    "No hay intervención en el periodo":99
}


In [ ]:
t_42 = {
    "si":1,
    "no":2,
    "Persona con aseguramiento":55,
    "Desconocido":99
}

In [ ]:
t_44 = {
    "Persona con aseguramiento":55
}

"""
Registre la enfermedad maligna
antecedente o concurrente. Tenga en cuenta que antes de definir, por ejemplo, un linfoma, el
usuario pudo tener un diagnóstico de cáncer a estudio o tumor de células pequeñas, redondas y
azules, debe notificarse el linfoma. Registre el código de la enfermedad maligna diagnosticada al
usuario según código CIE -10 (disponible en la página web CAC).

Registre 55= Persona con aseguramiento (régimen subsidiado o contributivo y que no son PPNA)
que recibió servicios de salud por parte del ente territorial durante el periodo de reporte.
Registre 99= No Aplica (no hay antecedente o concurrencia de otro cáncer primario)
"""

In [ ]:
t_45 = {
    "si":1,
    "No aplica":98,
    "persona con aseguramiento" : 55
}

In [ ]:
t_46 = {
    "numero de fases":None,
    "Es cáncer hematolinfático con los siguientes códigos de clasificación diagnóstica CIE 10 ( C835,C910, C920, C924 y C925)":0,
    "persona con aseguramiento":55,
    "No aplica":98
}

"""
Escriba el número
de fases de quimioterapia propuestas para este periodo.
0= Es cáncer hematolinfático con los siguientes códigos de clasificación diagnóstica CIE 10 ( C835,
C910, C920, C924 y C925) y en la variable 45 respondió la opción 2 ó 98
55= Persona con aseguramiento (régimen subsidiado o contributivo y que no son PPNA) que recibió
servicios de salud por parte del ente territorial durante el periodo de reporte.
98= No Aplica (es sólido o es cáncer diferente a los enunciados en las fases)
"""

In [ ]:
t_45 = {
    
}

In [2]:
import pandas as pd

data = pd.read_csv('municipios.csv',sep=',')
def lower(text):
    return text.lower()
def normalize(s):
    replacements = (
        ("á", "a"),
        ("é", "e"),
        ("í", "i"),
        ("ó", "o"),
        ("ó", "o"),
        ("ú", "u"),
    )
    for a, b in replacements:
        s = s.replace(a, b)
    return s


In [3]:
municipio = "riohacha"
data['municipio'] = data['municipio'].apply(lambda x: lower(x))
data['municipio'] = data['municipio'].apply(lambda x: normalize(x))
codigo = data[data['municipio']==municipio]['codigo'].values[0]

In [21]:
for i in range(len(data)):
    if municipio == data.loc[i,:].values[0]:
        codigo = data.loc[i,:].values[1]
    

In [26]:
import pandas as pd
eps = pd.read_csv('eps.csv',sep=',')

In [27]:
eps['eps'] = eps['eps'].apply(lambda x: lower(x))
eps['eps'] = eps['eps'].apply(lambda x: normalize(x))

In [28]:
len(eps)

85

In [29]:
eps.value_counts()

codigo  eps                                            
85000   direccion departamental de salud de casanare       1
EPSI03  asociacion indigena del cauca -a.i.c.              1
EPSS41  nueva eps s.a.                                     1
EPSS40  savia salud                                        1
EPSS34  capital salud epss sas.                            1
                                                          ..
EMP025  comfenalco valle empresa medicina prepagada        1
EMP023  colsanitas empresa medicina prepagada              1
EMP019  colpatria empresa medicina prepagada               1
EMP017  colmedica empresa medicina prepagada               1
REUE09  unidad de salud de la universidad del atlantico    1
Length: 85, dtype: int64

In [50]:
eps[eps.duplicated('eps')]


,codigo,eps
30,EMP029,colpatria empresa medicina prepagada
34,EPSS03,cafesalud eps
49,EPSS33,saludvida s.a. entidad promotora de salud
53,EPSS41,nueva eps s.a.


In [33]:

ANTIOQUIA               x3
BARRANQUILLA            x1
bogota                  x1
bolivar                 x1
CARTAGENA               x4
BOYACA                  x2
CALDAS                  x1
caqueta                 x1
CAUCA                   x3

,codigo,eps


SyntaxError: invalid syntax (<ipython-input-51-3a80a496a785>, line 1)